In [12]:
import cv2
import pickle
import numpy as np
import os

# Capturing video from the internal webcam
video = cv2.VideoCapture(0)

# Loading the pre-trained face detection model (Haar Cascade)
facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces_data = []
name = input("Enter your name please:")

Enter your name please: Adrish


gray: This is the input image in grayscale. The face detection algorithm typically works on grayscale images because color information is not necessary for detecting face structures.

1.3: This is the scale factor. It specifies how much the image size is reduced at each image scale. A value of 1.3 means the image is reduced by 30% for each pyramid layer. This helps the algorithm detect faces at different scales (sizes) within the image. Smaller values (close to 1) mean finer detection but at the cost of speed.

5: This is the minNeighbors parameter. It specifies how many neighbors each candidate rectangle (a detected face) should have to retain it. A higher value (like 5) means that the algorithm is more selective, reducing false positives by requiring that a face must be surrounded by a certain number of other detected objects (potential faces) to be considered valid.

In [8]:
i = 0
while True:
    #ret denotes camera is ok or not 
    ret,frame = video.read() 
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #converting
    
    # Detecting faces in the grayscale image
    faces = facedetect.detectMultiScale(gray,1.3,5)
    
    for (x,y,w,h) in faces:#it encoded rectangle
        crop_image = frame[y:y+h,x:x+w,:] #(row,colmn,channel)
        resized_image = cv2.resize(crop_image,(50,50))
        if len(faces_data) < 100 and i%10 ==0:
            faces_data.append(resized_image)
        i = i + 1 
        cv2.putText(frame,str(len(faces_data)),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(50,50,255),2)
        
        #cv2.rectangle(image, top-left, bottom-right, color(in BGR format), thickness)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(50,50,255),4)
    cv2.imshow("frame",frame) #window_name, material "to view"
    k = cv2.waitKey(1)
    if k == ord('q') or len(faces_data)== 100: # press q to quite
        break
print(len(faces_data))
video.release()
cv2.destroyAllWindows()

100


In [117]:
faces_data1 = np.asarray(faces_data)
print(faces_data1.shape)

#each 100 images flatten into a rows actually
faces_data2 = faces_data1.reshape(100,-1) 
print(faces_data2.shape)

(100, 50, 50, 3)
(100, 7500)


In [119]:
# Handling names
#creating first time when there is no pickle file of names id there
if 'names.pkl' not in os.listdir('data/'):
    names = [name] * 100
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)
        
#file already exist rewrite on file from last time where i stop
else:
    with open('data/names.pkl', 'rb') as f:
        names = pickle.load(f)
    names += [name] * 100  # Append new names
    with open('data/names.pkl', 'wb') as f:
        pickle.dump(names, f)

#if collection of faces_dataset is not there in locally then create andd write down from starting
if 'faces_data.pkl' not in os.listdir('data/'):
    with open('data/faces_data.pkl','wb') as f:
        pickle.dump(faces_data2,f)

#if collection of faces_datset is already exists
else:
    with open('data/faces_data.pkl','rb') as f:
        faces = pickle.load(f)
    faces = np.append(faces,faces_data2, axis = 0) #storing data after flatten all the 100 images (100, 50, 50, 3) to (100, 7500) 
    with open('data/faces_data.pkl','wb') as f:
        pickle.dump(faces,f)
        